<img src="images/prophet_model.png">

In [1]:
import os
from os import listdir
from os.path import isfile, join
from fbprophet import Prophet
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
def split_date(dataframe, date_col):
    df = dataframe.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df["year"] = df[date_col].dt.year - 2015
    df["month"] = df[date_col].dt.month
    df["week"] = df[date_col].dt.week
    df["day"] = df[date_col].dt.day
    df['quarter'] = df[date_col].dt.quarter
    df["day_of_week"] = df[date_col].dt.weekday
    df["weekend"] = (df["day_of_week"] > 5).astype(int)
    return df


In [3]:
train_path = "./train/"
test_path = "./test/"
pred_path = "./pred/"
train_prefix = "simple_train__"
test_prefix = "simple_test__"
pred_prefix = "pred_"


In [4]:
def get_files(suffixes, path, prefix):
    return [path + prefix + suffix for suffix in suffixes]

In [8]:
def predict(train_1_1, test_1_1):
    train_1_1 = train_1_1.sort_values("date")
    test_1_1 = test_1_1.sort_values("date")
    test_1_1["sales"] = np.nan
    city = test_1_1.loc[0, "city"]
    med = test_1_1.loc[0, "medicine"]
    df_final = pd.concat([train_1_1[["date", "sales", "discounted", "footfall", "imputed"]], 
                          test_1_1[["date", "sales", "discounted", "footfall", "imputed"]]], ignore_index=True)
    df_final = split_date(df_final, "date")
    df_final = df_final.rename({"date":"ds", "sales":"y"}, axis = 'columns')
    model = Prophet(interval_width=0.95)
    model.add_regressor("year", standardize=True)
    model.add_regressor("month", standardize=True)
    model.add_regressor("day", standardize=True)
    model.add_regressor("week", standardize=True)
    model.add_regressor("day_of_week", standardize=True)
    model.add_regressor("quarter", standardize=True)
    model.add_regressor("weekend", standardize=False)
    model.add_regressor("discounted", standardize=False)
    model.add_regressor("footfall", standardize=True)
    model.add_regressor("imputed", standardize=False)

    model.fit(df_final[df_final["ds"] < "2018-07-01"])
    future = df_final[['ds','discounted', 'footfall', 'imputed', 'weekend'
                       , 'year', 'month', 'week', 'day', 'quarter', 'day_of_week'
                      ]]
    forcast = model.predict(future)
    test_final = forcast[forcast["ds"] >= "2018-07-01"][["ds", "yhat"]]
    test_final["city"] = city
    test_final["medicine"] = med
    test_final = test_final.rename({"ds":"date", "yhat": "sales"}, axis='columns')
    return test_final


In [9]:
def get_predictions(train_path, train_prefix, test_path, test_prefix, pred_path, pred_prefix):
    trainfiles = [f[len(train_prefix):] for f in listdir(train_path) if isfile(join(train_path, f)) and train_prefix in f]
    testfiles = [f[len(test_prefix):] for f in listdir(test_path) if isfile(join(test_path, f)) and test_prefix in f]
    common_city_meds = set(testfiles).intersection(trainfiles)
    
    common_train_files = get_files(common_city_meds, train_path, train_prefix)
    common_test_files = get_files(common_city_meds, test_path, test_prefix)
    common_pred_files = get_files(common_city_meds, pred_path, pred_prefix)
    
    for i in range(0, len(common_test_files)):
        train_df = pd.read_csv(common_train_files[i])
        test_df = pd.read_csv(common_test_files[i])
        prediction = predict(train_df, test_df)
        prediction.to_csv(common_pred_files[i], index=False)

In [10]:
get_predictions(train_path, train_prefix, test_path, test_prefix, pred_path, pred_prefix)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

FileNotFoundError: [Errno 2] No such file or directory: './pred/pred_4_2745.csv'